In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotnine as p9
import os

from lib.lib import Import_data

path = Import_data()
# get all files in the directory
files = os.listdir(path)
print(files)

Path to dataset files: /home/chaotic/.cache/kagglehub/datasets/arashnic/earthquake-magnitude-damage-and-impact/versions/6
['csv_building_damage_assessment.csv', 'csv_building_ownership_and_use.csv', 'csv_building_structure.csv', 'csv_household_demographics.csv', 'csv_household_earthquake_impact.csv', 'csv_household_resources.csv', 'csv_individual_demographics.csv', 'mapping.csv', 'ward_vdcmun_district_name_mapping.csv']


In [11]:
building_types = {
    "Traditional Bamboo": {
        "description": "Simple bamboo structure with timber framing",
        "foundation_type": "Bamboo/Timber",
        "roof_type": "Bamboo/Timber-Light roof",
        "ground_floor_type": "Mud",
        "other_floor_type": "Not applicable",
        "has_superstructure_timber": 1,
        "has_superstructure_bamboo": 1,
        "base_cost": 909,  # 1 LAKH NPR
        "src": "/building_types/bamboo.png",
    },
    "Basic Stone/Brick": {
        "description": "Stone/brick home with light roof and mud mortar",
        "foundation_type": "Mud mortar-Stone/Brick",
        "roof_type": "Bamboo/Timber-Light roof",
        "ground_floor_type": "Mud",
        "other_floor_type": "TImber/Bamboo-Mud",
        "has_superstructure_mud_mortar_stone": 1,
        "base_cost": 1455,  # NPR 100,000 to NPR 500,000
        "src": "/building_types/brick_mud.png",
    },
    "Improved Stone/Brick": {
        "description": "Stone/brick home with heavy timber roof for better stability",
        "foundation_type": "Mud mortar-Stone/Brick",
        "roof_type": "Bamboo/Timber-Heavy roof",
        "ground_floor_type": "Brick/Stone",
        "other_floor_type": "TImber/Bamboo-Mud",
        "has_superstructure_mud_mortar_stone": 1,
        "base_cost": 3637,
        "src": "/building_types/improved_brick.png",
    },
    "Modern Concrete": {
        "description": "Reinforced concrete structure with modern materials",
        "foundation_type": "RC",
        "roof_type": "RCC/RB/RBC",
        "ground_floor_type": "RC",
        "other_floor_type": "RCC/RB/RBC",
        "has_superstructure_cement_mortar_brick": 1,
        "base_cost": 7275,
        "src": "/building_types/modern_concrete.png",
    },
}


In [12]:
building_structure = pd.read_csv(path + "/csv_building_structure.csv")

In [15]:


def get_buildings(building):
    foundation_type = building_structure["foundation_type"] == building["foundation_type"]
    roof_type = building_structure["roof_type"] == building["roof_type"]
    ground_floor_type = building_structure["ground_floor_type"] == building["ground_floor_type"]
    other_floor_type = building_structure["other_floor_type"] == building["other_floor_type"]
    has_superstructure_mud_mortar_stone = building_structure["has_superstructure_mud_mortar_stone"] == building.get("has_superstructure_mud_mortar_stone", 0)
    has_superstructure_timber = building_structure["has_superstructure_timber"] == building.get("has_superstructure_timber", 0)
    has_superstructure_bamboo = building_structure["has_superstructure_bamboo"] == building.get("has_superstructure_bamboo", 0)
    has_superstructure_cement_mortar_brick = building_structure["has_superstructure_cement_mortar_brick"] == building.get("has_superstructure_cement_mortar_brick", 0)
    df = building_structure[
        foundation_type
        & roof_type
        & ground_floor_type
        & other_floor_type
        & has_superstructure_mud_mortar_stone
        & has_superstructure_timber
        & has_superstructure_bamboo
        & has_superstructure_cement_mortar_brick
    ]
    return df


def calculate_damage_grade(building_structure, text):
    
    # Calculate damage grade
    damage_grades = building_structure["damage_grade"].copy()
# Weighted average by count damage grade
    damage_grade = damage_grades.value_counts().sort_index()
    damage_grade.index = np.array([1, 2, 3, 4, 5])
    damage_grade = damage_grade / damage_grade.sum()
    damage_grade = damage_grade * damage_grade.index
    damage_grade = damage_grade.sum()
    print("Building Type: ", text)
    print("Damage Grade: ", damage_grade)

for building_name, building in building_types.items():
    building = building_types[building_name]
    structures = get_buildings(building)
    calculate_damage_grade(structures, building_name)

Building Type:  Traditional Bamboo
Damage Grade:  2.4688940092165894
Building Type:  Basic Stone/Brick
Damage Grade:  4.005705185903206
Building Type:  Improved Stone/Brick
Damage Grade:  4.031253052055865
Building Type:  Modern Concrete
Damage Grade:  1.6486562942008485


In [14]:
damage_grades.value_counts()

NameError: name 'damage_grades' is not defined